In [1]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import csv
import numpy as np
from numba import jit
import os


def plot_grid(grid, img_id):
    print(f"Plotting grid for {img_id}")
    grid = get_wb_borders(grid)
    fig, ax = plt.subplots()
    ax.imshow(grid, cmap="gray", interpolation="nearest")
    ax.axis("off")  # Hide the axes
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Remove padding
    filename = f"output/{img_id}.png"
    fig.savefig(filename, bbox_inches="tight", pad_inches=0)
    plt.close(fig)


def save_grid(grid, img_id, size):
    grid = get_wb_borders(grid)
    directory = f"output{size}"
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(f"output{size}/{img_id}.csv", mode="a", newline="") as file:
        writer = csv.writer(file)
        for i in range(grid.shape[0]):
            writer.writerow(grid[i])


@jit(nopython=True)
def get_wb_borders(grid):
    size = grid.shape[0]
    wb_array = np.zeros_like(grid, dtype=np.uint8)

    for x in range(1, size - 1):
        for y in range(1, size - 1):
            current_value = grid[x, y]
            if (
                grid[x - 1, y] != current_value
                or grid[x + 1, y] != current_value
                or grid[x, y - 1] != current_value
                or grid[x, y + 1] != current_value
            ):
                wb_array[x, y] = 1

    return wb_array


def plot_combined(temperature, grid, filename):
    grid = get_wb_borders(grid)

    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    # Plot temperature
    axs[0].imshow(temperature, cmap="hot", interpolation="nearest")
    axs[0].set_title("Temperature")
    fig.colorbar(axs[0].images[0], ax=axs[0])

    # Create a unique color map for the grid
    unique_values = np.unique(grid)
    num_unique_values = len(unique_values)
    colors = plt.cm.get_cmap(
        "tab20", num_unique_values
    ).colors  # Generate a color for each unique value
    cmap = ListedColormap(colors)

    # Plot grid
    cax = axs[1].imshow(grid, cmap=cmap, interpolation="nearest")
    axs[1].set_title("Grid")

    # Create a colorbar with the correct labels
    cbar = fig.colorbar(cax, ax=axs[1], ticks=unique_values)
    cbar.ax.set_yticklabels(
        [str(val) for val in unique_values]
    )  # Set the ticks to be the unique values

    # Save the combined plot as a PNG file
    # plt.savefig(filename)
    plt.show()

In [ ]:
import numpy as np
from numba import jit
import csv
import uuid


@jit(nopython=True)
def random_grain(prob, max_val):
    return np.random.randint(1, max_val + 1) if np.random.random() < prob else 0


@jit(nopython=True)
def update_grid_numba(grid, temperature, dt):
    hasLiquid = False
    new_grid = grid.copy()
    size = grid.shape[0]
    for x in range(1, size - 1):
        for y in range(1, size - 1):
            if temperature[x, y] > 0.80:
                new_grid[x, y] = 0
                continue
            if grid[x, y] == 0:
                hasLiquid = True
                prob = (1 - temperature[x, y] / 0.8) / 500
                if np.random.random() < prob:
                    new_grid[x, y] = np.random.randint(1, 20)
                neighbors = grid[x - 1 : x + 2, y - 1 : y + 2].ravel()
                non_zero_neighbors = neighbors[neighbors != 0]
                if len(non_zero_neighbors) > 0 and np.random.random() > (
                    temperature[x, y] / 0.8
                ):
                    new_grid[x, y] = np.random.choice(non_zero_neighbors)
    return new_grid, hasLiquid


@jit(nopython=True)
def update_temp_2d_numba(u, dt, dx, dy, k):
    u_new = u.copy()
    nx, ny = u.shape

    y1 = u_new.shape[1] - 1
    for x1 in range(u_new.shape[0]):
        u_new[x1, y1] = min(u_new[x1, y1 - 1], u_new[x1, y1 - 2])
        u_new[x1, 0] = min(u_new[x1, 1], u_new[x1, 2])
    x2 = u_new.shape[0] - 1
    for y2 in range(u_new.shape[1]):
        u_new[x2, y2] = min(u_new[x2 - 1, y2], u_new[x2 - 2, y2])
        u_new[0, y2] = min(u_new[1, y2], u_new[2, y2])

    for i in range(1, nx - 1):
        for j in range(1, ny - 1):
            u_new[i, j] = u[i, j] + k * dt * (
                (u[i + 1, j] - 2 * u[i, j] + u[i - 1, j]) / dx**2
                + (u[i, j + 1] - 2 * u[i, j] + u[i, j - 1]) / dy**2
            )
    return u_new


@jit(nopython=True)
def apply_heat_numba(u, x, y, r, initial_temp, air_temp):
    u_new = u.copy()
    nx, ny = u.shape
    for i in range(1, nx - 1):
        for j in range(1, ny - 1):
            dist_sq = (i - x) ** 2 + (j - y) ** 2
            if dist_sq <= r**2:
                u_new[i, j] = initial_temp - (initial_temp - air_temp) * (
                    dist_sq / (r * r)
                )
    return u_new


def mainLoop(img_id, size, initX, initY):
    points = [(initX, initY, 0)]
    dx, dy = 0.1, 0.1
    dt = 0.01
    time = 0
    end_time = 1.7
    initial_temp = 2
    air_temp = 0.01
    k = 1

    if dt > 1 / (2 * k * (1 / dx / dx + 1 / dy / dy)):
        dt = 1 / (2 * k * (1 / dx / dx + 1 / dy / dy))
        print(f"Adjusted dt for stability. New value dt={dt}")

    nx = ny = size
    u = air_temp * np.ones((nx + 2, ny + 2))
    u[0, :] = u[:, 0] = u[-1, :] = u[:, -1] = air_temp

    grid = np.zeros((size * 4 + 2, size * 4 + 2), dtype=np.uint8)
    for i in range(1, size * 4 + 1):
        for j in range(1, size * 4 + 1):
            grid[i, j] = random_grain(1 / 500, 19)

    n = 0
    dtCA = 0.01
    timeCA = 0
    timeHeat = -1

    hasLiquid = True

    while hasLiquid:
        time += dt
        n += 1
        while points and points[0][2] < time:
            lastHeatX, lastHeatY, _ = points.pop(0)
            timeHeat = time
            u = apply_heat_numba(
                u, lastHeatX + 1, lastHeatY + 1, size // 2, initial_temp, air_temp
            )
            grid, hasLiquid = update_grid_numba(
                grid, np.repeat(np.repeat(u, 4, axis=0), 4, axis=1), dt
            )

        if time - timeCA > dtCA:
            timeCA += dtCA
            grid, hasLiquid = update_grid_numba(
                grid, np.repeat(np.repeat(u, 4, axis=0), 4, axis=1), dt
            )

        # if n % 10 == 0:
        # print(f"Simulated for time {time:.2f}")

        u = update_temp_2d_numba(u, dt, dx, dy, k)

    # print(f"Simulation completed for {img_id}")

    # plot_grid(grid, img_id)
    # plot_combined(u, grid, f"output/{img_id}.png")
    save_grid(grid, img_id, size)


def main():
    size = 32
    samples = []

    genlen = 10000
    initXs = np.random.randint(0, size, genlen)
    initYs = np.random.randint(0, size, genlen)
    img_ids = [uuid.uuid4() for _ in range(genlen)]

    for i in range(genlen):
        initX = initXs[i]
        initY = initYs[i]
        img_id = img_ids[i]
        samples.append([initX, initY, img_id])
        mainLoop(img_id, size, initX, initY)
        print(f"Simulation completed for {i}")

    with open(f"input{size}.csv", mode="a", newline="") as file:
        writer = csv.writer(file)
        for i in range(len(samples)):
            writer.writerow(samples[i])


if __name__ == "__main__":
    main()